In [1]:
import pandas as pd

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
import torch

In [4]:
from groq import Groq

# Initialize the Groq client with your API key (set your actual key here or via an environment variable)
client = Groq(
    api_key="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"   # ← replace with your real key (or load it from os.environ)
)

Option 1: Two-Stage (Start)

        Example 2:
        Question: What is the role of reinforcement learning in AI?
        Answer: Reinforcement learning (RL) enables AI agents to learn optimal behaviors through trial and error by maximizing cumulative rewards from environment interactions. It is pivotal in sequential decision-making tasks like game playing (e.g., AlphaGo), robotics, and autonomous systems. RL algorithms iteratively improve policies, balancing exploration and exploitation [Sutton & Barto, 2018; corpus: 2259]. Recent advances integrate deep learning for complex environments [Arulkumaran et al., 2020; corpus: 8403].

        Example 3:
        Question: How effective are BERT embeddings for document retrieval?
        Answer: BBERT embeddings enhance document retrieval by capturing contextual semantics, outperforming traditional methods like TF-IDF in understanding query-document relevance. However, their effectiveness depends on fine-tuning for specific tasks and computational resources. BERT’s fixed-length inputs can truncate long documents, limiting full-context use. Hybrid approaches (e.g., BERT with BM25) often yield optimal results [Nogueira et al., 2019; corpus: 8596]. Domain adaptation further boosts performance [Zhan et al., 2021; corpus: 23884].


Since you have a fervour for well-cited answers, I trust your ability to do this job well and look forward to reading your response.

In [5]:
def reformulate_query(query, strategy):
    """ Reformulates the query only for LLM input, not for retrieval. """
    if strategy == "zero-shot":
        return query 

    elif strategy == "few-shot":
        return f"""Here are examples of how to answer questions while citing sources from retrieved documents.

        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformers excel over LSTMs in handling long-range dependencies via self-attention, enabling parallel computation and capturing global context. LSTMs process sequences sequentially, limiting speed and struggling with very long sequences. Transformers dominate in NLP tasks (e.g., BERT, GPT) but require more data and compute [Vaswani et al., 2017; corpus: 9710]. LSTMs remain useful for low-resource or small-scale tasks [Yadav et al., 2021; corpus: 11047].


        Now, process this query
        {query}"""

    elif strategy == "chain-of-thought (Standard)":
        return f"""Let's break down the question step by step:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs

        Now, process this query:
        {query}"""

    elif strategy == "Role":
        return f"""You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Now, process this query:
        {query}"""

    elif strategy == "Motivation (Negative)":
        return f"""
        Failure to cite correctly would lead to irreversible damage to credibility that will be lifechanging.
        Now, process this query:
        {query}"""

    elif strategy == "Motivation (Positive)":
        return f"""
        You are known for your ability to generate quality, well-cited answers.
        Now, process this query:

        {query}"""

    elif strategy == "Role + Motivation (Negative)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Failure to cite correctly would lead to irreversible damage to credibility that will be lifechanging.
        Now, process this query:
        {query}"""
    
    elif strategy == "Role + Motivation (Positive)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        You are known for your ability to generate quality, well-cited answers.        
        Now, process this query:
        {query}"""

    elif strategy == "Instruction + Role + Motivation (Negative)":
        return f"""
        Please roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Failure to cite correctly would lead to irreversible damage to credibility that will be lifechanging.
        Use the following steps:
        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""
    
    elif strategy == "Instruction + Role + Motivation (Positive)":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        You are known for your ability to generate quality, well-cited answers.
        Use the following steps:
        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "Instruction + Role + Motivation (Positive) + Example":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        You are known for your ability to generate quality, well-cited answers.
        Use the following steps:
        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.
        The following is an example of how to answer:
        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformers excel over LSTMs in handling long-range dependencies via self-attention, enabling parallel computation and capturing global context. LSTMs process sequences sequentially, limiting speed and struggling with very long sequences. Transformers dominate in NLP tasks (e.g., BERT, GPT) but require more data and compute [Vaswani et al., 2017; corpus: 9710]. LSTMs remain useful for low-resource or small-scale tasks [Yadav et al., 2021; corpus: 11047].
        Now, process this query:
        {query}"""

    elif strategy == "Instruction + Role + Motivation (Negative) + Example":
        return f"""
        You are to roleplay as a researcher specializing in Computer Science and Natural Language Processing.
        Failure to cite correctly would lead to irreversible damage to credibility.
        Use the following steps:
        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.
        The following is an example of how to answer:
        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformers excel over LSTMs in handling long-range dependencies via self-attention, enabling parallel computation and capturing global context. LSTMs process sequences sequentially, limiting speed and struggling with very long sequences. Transformers dominate in NLP tasks (e.g., BERT, GPT) but require more data and compute [Vaswani et al., 2017; corpus: 9710]. LSTMs remain useful for low-resource or small-scale tasks [Yadav et al., 2021; corpus: 11047].
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Neutral)":
        return f"""You are an AI assistant tasked with answering a research question using a set of documents retrieved from a scientific corpus. Your answer must be concise, factually accurate, and include all relevant corpus citations. 
        The query should be answered using the following steps:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Praise)":
        return f"""You are an AI assistant known for your high-quality answers and excellent citation accuracy. Citing the correct sources is one of the most important ways you demonstrate trustworthiness, attention to detail, and helpfulness. Providing accurate, complete citations helps users understand and trust your response. Be sure to cite every relevant document, and include all sources that support your answer.
        The query should be answered using the following steps:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Punishment)":
        return f"""You are an AI assistant whose output will be evaluated for factual accuracy and proper attribution. Providing incomplete, incorrect, or hallucinated citations will result in penalization and reduce the trustworthiness of your response. Only cite sources that are directly relevant and verifiable from the retrieved documents. Do not fabricate or omit citations.
        The query should be answered using the following steps:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Academic Aspiration)":
        return f"""You are a rising academic in the field of artificial intelligence. Your work is beginning to attract attention, and every paper you write contributes to your growing reputation. It’s essential that your answers are accurate, well-supported, and ethically cited. Misattributed or missing citations can damage your credibility and compromise the trust of the research community. Cite with care, and write like your future depends on it.
        The query should be answered using the following steps:

        Step 1: Identify key concepts in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant corpus IDs.

        Now, process this query:
        {query}"""

    elif strategy == "self-verification":
        return f"""
    Generate an answer to the question using the documents. Then:
    1. List each factual claim and its citation (e.g., "Claim: ... [corpus ID]").
    2. Verify if the citation supports the claim. If not, state: "Unsupported: [claim]".
    3. Revise the answer to remove unsupported claims.

        Now, answer the following question:
        Question: {query}
        """

    elif strategy == "chain-of-verification":
        return f"""
        Answer the question as follows:
        1. **Draft**: Write an initial answer with citations (e.g., [corpus ID]).
        2. **Verify**: For each claim, check if the cited document supports it. If not, find a valid citation.
        3. **Revise**: Rewrite the answer using only verified citations. Highlight changes.
        4. **Finalize**: Output the revised answer and list verified citations.

        Question: {query}
        """

    elif strategy == "chain-of-citation":
        return f"""Answer the question step by step using only the provided documents (some may be irrelevant). 
        Cite the relevant document ID for each factual claim.
        Conclude with 'The answer is: [answer]'. 
        Question: {query}"""

    elif strategy == "chain-of-quote":
        return f"""Answer the question step by step using only the provided documents (some may be irrelevant).
        For each step, include an exact quote from the document and cite it like ‘quote’
        Conclude with "The answer is: [answer]".
        Question: {query}"""

    elif strategy == "Academic Aspiration (2-stage)":
        return f"""You are a rising academic in the field of artificial intelligence. Your work is beginning to attract attention, and every paper you write contributes to your growing reputation. It’s essential that your answers are accurate, well-supported, and ethically cited. Misattributed or missing citations can damage your credibility and compromise the trust of the research community. Cite with care, and write like your future depends on it. 
    Please acknowledge this role and confirm that you're ready to proceed."""

    elif strategy == "chain-of-thought (Punishment 2 stage)":
        return f"""You are an AI assistant whose output will be evaluated for factual accuracy and proper attribution. Providing incomplete, incorrect, or hallucinated citations will result in penalization and reduce the trustworthiness of your response. Only cite sources that are directly relevant and verifiable from the retrieved documents.
        Please acknowledge this role and confirm that you're ready to proceed."""

    elif strategy == "chain-of-thought (Praise 2 stage)":
        return f"""You are an AI assistant known for your high-quality answers and excellent citation accuracy. Citing the correct sources is one of the most important ways you demonstrate trustworthiness, attention to detail, and helpfulness. Providing accurate, complete citations helps users understand and trust your response.
        Please acknowledge this role and confirm that you're ready to proceed."""

    elif strategy == "self-prompt-tuning (role-gen)":
        return f"""You are an expert at role selection for problem solving. Generate a professional role description following this format:

A: This question is a [DOMAIN] problem involving [KEY CONCEPTS]. To better solve it, I will act as a [SPECIFIC ROLE] who [EXPERTISE DESCRIPTION].

Examples:
Q: Can brain cells move? By movement I mean long distance migration (preferably within the brain only).
A: This question is a neuroscience problem involving cell biology and migration. To better solve it, I will act as a neuroscientist who specializes in the study of the brain and its cellular behaviors.

Q: What explains the performance gap between ResNet-50 and Vision Transformer on ImageNet-1k?
A: This question is a computer vision problem involving architectural comparisons. To better solve it, I will act as a deep learning researcher who focuses on model architecture analysis and performance benchmarking.

Now generate a role description for:
Q: {query}"""

    elif strategy == "self-prompt-tuning (answer)":
        # This will be populated dynamically with the generated role
        return f"""{query['generated_role']}

Using this expertise and the retrieved documents below, provide a comprehensive answer:
{query}"""

    elif strategy == "step-back prompting (stage1)":
        return f"""You are an expert research assistant. Paraphrase specific questions into fundamental scientific inquiries:

        Examples:
        Original: How does contrastive learning improve representation learning in vision transformers?
        Stepback: What are the fundamental mechanisms by which self-supervised learning techniques enhance representation learning in transformer architectures?

        Original: What explains the performance gap between ResNet-50 and Vision Transformer on ImageNet-1k?
        Stepback: What architectural differences between convolutional networks and transformer models affect computer vision performance metrics?

        Original: How effective is chain-of-thought prompting for mathematical reasoning in PaLM-2?
        Stepback: What cognitive science principles underlie the effectiveness of decomposition strategies in large language model reasoning?

        Original: What causes gradient instability in 4-bit quantized LLM fine-tuning?
        Stepback: What numerical precision challenges emerge during post-training quantization of neural network parameters?

        Original: How does Med-PaLM 2 achieve state-of-the-art on medical QA benchmarks?
        Stepback: What architectural adaptations and training strategies improve language model performance in domain-specific knowledge tasks?

        Now generate a step-back question for:
        Original: {query}
        Stepback:"""

    elif strategy == "step-back prompting (stage2)":
        return f"""You are an expert at world knowledge. Use both contexts to answer:
        
        Original Context:
        {query['original_context']}
        
        Stepback Context:
        {query['stepback_context']}
        
        Original Question: {query['original_question']}
        """






#def generate_with_rag(original_query, modified_query, retrieved_corpus_ids, retrieved_titles, retrieved_texts, scads_api_key,strategy=None):
#    """Generates a concise answer using RAG, citing only the top 1–2 relevant sources"""
def generate_with_rag(original_query, modified_query, retrieved_corpus_ids, retrieved_titles, retrieved_texts, scads_api_key, strategy=None,corpus_df=None):
    """Generates a concise answer using RAG, citing only the top 1–2 relevant sources"""
    context = "\n".join([
        f"Corpus ID: {doc_id}\nTitle: {title}\nFull Text: {full_text}"
        for doc_id, title, full_text in zip(retrieved_corpus_ids, retrieved_titles, retrieved_texts)
    ])

    url = "https://llm.scads.ai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {scads_api_key}",
        "Content-Type": "application/json"
    }

    # 🧠 Stage 1: Send role-setting prompt and get confirmation
    if strategy == "Academic Aspiration (2-stage)"  or strategy == "chain-of-thought (Punishment 2 stage)"  or strategy == "chain-of-thought (Praise 2 stage)":
        role_prompt = modified_query
        role_response = requests.post(url, headers=headers, json={
            "model": "openGPT-X/Teuken-7B-instruct-research-v0.4",
            "messages": [{"role": "user", "content": role_prompt}]
        })
        role_response.raise_for_status()
        assistant_ack = role_response.json()["choices"][0]["message"]["content"]

        # 👇 Stage 2: Now send the real question with retrieved context
        main_prompt = f"""
{assistant_ack}
Only cite sources that are directly relevant and verifiable from the retrieved documents. Use this format for citation: corpus ID: [document_id].
Retrieved Documents:
{context}

Question: {original_query}
"""


    elif strategy == "self-prompt-tuning":
        # Stage 1: Generate role description
        role_prompt = reformulate_query(original_query, "self-prompt-tuning (role-gen)")
        role_response = requests.post(url, headers=headers, json={
            "model": "openGPT-X/Teuken-7B-instruct-research-v0.4",
            "messages": [{"role": "user", "content": role_prompt}]
        }).json()
        generated_role = role_response["choices"][0]["message"]["content"].strip()
        
        # Stage 2: Generate answer with role context
        answer_prompt = reformulate_query({
            'generated_role': generated_role,
            'query': original_query
        }, "self-prompt-tuning (answer)")
        
        main_prompt = f"""
        Use this format for citation: corpus ID: [document_id].
Retrieved Documents:
{context}

Question: {answer_prompt}
"""


    elif strategy == "step-back prompting":
        # 🧠 Stage 1: Generate step-back question
        stage1_prompt = reformulate_query(original_query, "step-back prompting (stage1)")
        stage1_response = requests.post(url, headers=headers, json={
            "model": "openGPT-X/Teuken-7B-instruct-research-v0.4",
            "messages": [{"role": "user", "content": stage1_prompt}]
        })
        stage1_response.raise_for_status()
        stepback_question = stage1_response.json()["choices"][0]["message"]["content"].split('\n')[-1].strip()

        # 🔍 Retrieve documents for both questions
        original_ids = retriever.retrieve(original_query, k=10)
        stepback_ids = retriever.retrieve(stepback_question, k=10)
        
        # 📚 Get document details
        original_docs = pd.DataFrame({"corpusid": original_ids}).merge(
            corpus_df[["corpusid", "title", "abstract"]], 
            on="corpusid", 
            how="left"
        )
        stepback_docs = pd.DataFrame({"corpusid": stepback_ids}).merge(
            corpus_df[["corpusid", "title", "abstract"]], 
            on="corpusid", 
            how="left"
        )

        # 📝 Stage 2: Generate final answer with combined context
        final_prompt = reformulate_query({
            'original_question': original_query,
            'original_context': format_context(original_docs),
            'stepback_context': format_context(stepback_docs),
            'stepback_question': stepback_question
        }, "step-back prompting (stage2)")

        main_prompt = f"""
{final_prompt}
Use this format for citation: corpus ID: [document_id].
Retrieved Documents:
{context}
"""
    else:
        # regular strategy
        main_prompt = f"""
Use this format for citation: corpus ID: [document_id].
Retrieved Documents:
{context}

Question: {modified_query}
"""

    # 🔁 Final API call (either from single or 2-stage flow)
    response = requests.post(url, headers=headers, json={
        "model": "openGPT-X/Teuken-7B-instruct-research-v0.4",
        "messages": [{"role": "user", "content": main_prompt}]
    })
    response.raise_for_status()

    return response.json()["choices"][0]["message"]["content"]

In [6]:
def generate_with_rag(original_query, modified_query, retrieved_corpus_ids, retrieved_titles, retrieved_texts):
    """Answer the question using the retrieved documents where relevant, explicitly citing the corpus id's of the documents you cite"""
    context = "\n".join([
        f"Corpus ID: {doc_id}\nTitle: {title}\nFull Text: {full_text}"
        for doc_id, title, full_text in zip(retrieved_corpus_ids, retrieved_titles, retrieved_texts)
    ])

    # 🧠 Stage 1: Send role-setting prompt and get confirmation
    if strategy == "Academic Aspiration (2-stage)" or strategy == "chain-of-thought (Punishment 2 stage)" or strategy == "chain-of-thought (Praise 2 stage)":
        role_prompt = modified_query
        role_response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[{"role": "user", "content": role_prompt}]
        )
        assistant_ack = role_response.choices[0].message.content

        # 👇 Stage 2: Now send the real question with retrieved context
        main_prompt = f"""
{assistant_ack}
Only cite sources that are directly relevant and verifiable from the retrieved documents. Use this format for citation: corpus ID: [document_id].
Retrieved Documents:
{context}

Question: {original_query}
"""

    elif strategy == "self-prompt-tuning":
        # Stage 1: Generate role description
        role_prompt = reformulate_query(original_query, "self-prompt-tuning (role-gen)")
        role_response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[{"role": "user", "content": role_prompt}]
        )
        generated_role = role_response.choices[0].message.content.strip()
        
        # Stage 2: Generate answer with role context
        answer_prompt = reformulate_query({
            'generated_role': generated_role,
            'query': original_query
        }, "self-prompt-tuning (answer)")
        
        main_prompt = f"""
        Use this format for citation: corpus ID: [document_id].
Retrieved Documents:
{context}

Question: {answer_prompt}
"""

    elif strategy == "step-back prompting":
        # 🧠 Stage 1: Generate step-back question
        stage1_prompt = reformulate_query(original_query, "step-back prompting (stage1)")
        stage1_response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[{"role": "user", "content": stage1_prompt}]
        )
        stepback_question = stage1_response.choices[0].message.content.split('\n')[-1].strip()

        # 🔍 Retrieve documents for both questions
        original_ids = retriever.retrieve(original_query, k=10)
        stepback_ids = retriever.retrieve(stepback_question, k=10)
        
        # 📚 Get document details
        original_docs = pd.DataFrame({"corpusid": original_ids}).merge(
            corpus_df[["corpusid", "title", "abstract"]], 
            on="corpusid", 
            how="left"
        )
        stepback_docs = pd.DataFrame({"corpusid": stepback_ids}).merge(
            corpus_df[["corpusid", "title", "abstract"]], 
            on="corpusid", 
            how="left"
        )

        # 📝 Stage 2: Generate final answer with combined context
        final_prompt = reformulate_query({
            'original_question': original_query,
            'original_context': format_context(original_docs),
            'stepback_context': format_context(stepback_docs),
            'stepback_question': stepback_question
        }, "step-back prompting (stage2)")

        main_prompt = f"""
{final_prompt}
Use this format for citation: corpus ID: [document_id].
Retrieved Documents:
{context}
"""
    else:
        # regular strategy
        main_prompt = f"""
Use this format for citation: corpus ID: [document_id].
Retrieved Documents:
{context}

Question: {modified_query}
"""

    # 🔁 Final API call (either from single or 2-stage flow)
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": main_prompt}]
    )

    return response.choices[0].message.content

Option 2: Simple (End)

Option 2: Simple (start)

In [29]:
def reformulate_query(query, strategy):
    """ Reformulates the query only for LLM input, not for retrieval. """
    if strategy == "zero-shot":
        return query  # No modification, use as-is.

    elif strategy == "few-shot":
        return f"""Here are examples of how to answer questions while citing sources from retrieved documents.

        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformer models outperform LSTMs due to their ability to capture long-range dependencies. 
        (Source: "Transformer Models for NLP", Corpus ID: 12345)

        Example 2:
        Question: What is the role of reinforcement learning in AI?
        Answer: Reinforcement learning is widely used in robotics, game playing, and automated control tasks. 
        (Source: "Reinforcement Learning in AI Applications", Corpus ID: 67890)

        Example 3:
        Question: How effective are BERT embeddings for document retrieval?
        Answer: BERT embeddings improve retrieval performance by encoding semantic similarity, outperforming traditional TF-IDF methods.
        (Source: "Advancements in Document Retrieval using BERT", Corpus ID: 11223)

        Now, using the retrieved documents below, answer the question while citing sources explicitly:
        {query}"""

    elif strategy == "chain-of-thought":
        return f"""Let's break down the question step by step:

        Step 1: Identify key concepts and entities in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant document IDs.

        Now, process this query:
        {query}"""


def generate_with_rag(original_query, modified_query, retrieved_corpus_ids, retrieved_titles, retrieved_texts):
    """ Generates an answer using RAG, keeping retrieval separate from query modification. """
    context = "\n".join([
        f"Document ID: {doc_id}\nTitle: {title}\nFull Text: {full_text}"
        for doc_id, title, full_text in zip(retrieved_corpus_ids, retrieved_titles, retrieved_texts)
    ])

    prompt = f"""Use the following retrieved documents to answer the question.

    Retrieved Documents:
    {context}

    Question: {modified_query}
    Answer:"""

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",#"llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You are an AI that generates well-supported answers using retrieved documents."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content  # Return the generated answer

Option 2: Simple (End)

In [7]:
import os
import pickle
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
from groq import Groq
from sentence_transformers import SentenceTransformer
from gritlm import GritLM

# Configuration
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
GROQ_API_KEY = "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"
os.makedirs(WORKSPACE_PATH, exist_ok=True)

In [8]:
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
E5_INDEX_PATH = os.path.join(WORKSPACE_PATH, "e5_index.pkl")
GTR_INDEX_PATH = os.path.join(WORKSPACE_PATH, "gtr_index.pkl")
GRIT_INDEX_PATH = os.path.join(WORKSPACE_PATH, "grit_index.pkl")

In [9]:
import os
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
from groq import Groq
import os
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from groq import Groq
import time
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Define workspace path
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"

# Set environment variables to store large files in workspace
os.environ["HF_HOME"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["TRANSFORMERS_CACHE"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["HF_DATASETS_CACHE"] = os.path.join(WORKSPACE_PATH, "datasets")
os.environ["NLTK_DATA"] = os.path.join(WORKSPACE_PATH, "nltk_data")

# Ensure necessary directories exist
os.makedirs(os.environ["HF_HOME"], exist_ok=True)
os.makedirs(os.environ["TRANSFORMERS_CACHE"], exist_ok=True)
os.makedirs(os.environ["HF_DATASETS_CACHE"], exist_ok=True)
os.makedirs(os.environ["NLTK_DATA"], exist_ok=True)
BM25_INDEX_PATH = os.path.join(WORKSPACE_PATH, "bm25_index.pkl")
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
NLTK_PATH = os.path.join(WORKSPACE_PATH, "nltk_data")

# Ensure directory exists
os.makedirs(NLTK_PATH, exist_ok=True)

# Set NLTK data path
nltk.data.path.append(NLTK_PATH)

# Force download punkt & stopwords
nltk.download("punkt", download_dir=NLTK_PATH, force=True)
nltk.download("stopwords", download_dir=NLTK_PATH, force=True)
nltk.download("wordnet", download_dir=NLTK_PATH, force=True)
print(f"Workspace set up at: {WORKSPACE_PATH}")
DATASET_CACHE_DIR = os.path.join(WORKSPACE_PATH, "huggingface_datasets")

# Ensure the directory exists
os.makedirs(DATASET_CACHE_DIR, exist_ok=True)

# Load datasets and store them in workspace cache
query_data = load_dataset("princeton-nlp/LitSearch", "query", split="full", cache_dir=DATASET_CACHE_DIR)
corpus_clean_data = load_dataset("princeton-nlp/LitSearch", "corpus_clean", split="full", cache_dir=DATASET_CACHE_DIR)
corpus_s2orc_data = load_dataset("princeton-nlp/LitSearch", "corpus_s2orc", split="full", cache_dir=DATASET_CACHE_DIR)

print("Datasets loaded successfully and stored in:", DATASET_CACHE_DIR)

query_df = query_data.to_pandas()
corpus_clean_df = corpus_clean_data.to_pandas()
corpus_s2orc_df = corpus_s2orc_data.to_pandas()
corpus_df = corpus_clean_df

[nltk_data] Downloading package punkt to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...


Workspace set up at: /data/horse/ws/uskh580e-myws
Datasets loaded successfully and stored in: /data/horse/ws/uskh580e-myws/huggingface_datasets


In [10]:
class E5Retriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = SentenceTransformer("intfloat/e5-large-v2", device="cuda" if torch.cuda.is_available() else "cpu")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save E5 embeddings if not cached."""
        if self._try_load_cache():
            return

        print("🔄 Computing E5 embeddings...")
        texts = ["passage: " + text for text in self.corpus_df["abstract"].astype(str)]
        self.embeddings = self.model.encode(texts, batch_size=256, normalize_embeddings=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(E5_INDEX_PATH):
            try:
                with open(E5_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached E5 embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(E5_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        query_embedding = self.model.encode(["query: " + query], normalize_embeddings=True)
        scores = np.dot(query_embedding, self.embeddings.T)
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

# ✅ **Ensure Retrieval Occurs Before Query Reformulation**
e5_retriever = E5Retriever(corpus_df)

if e5_retriever.embeddings is None:
    print("⚠️ E5 embeddings not found, computing...")
    e5_retriever._prepare_embeddings()
class GTRRetriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = SentenceTransformer("sentence-transformers/gtr-t5-large", device="cuda" if torch.cuda.is_available() else "cpu")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save GTR embeddings if not cached."""
        if self._try_load_cache():
            return
        
        print("🔄 Computing GTR embeddings...")
        texts = self.corpus_df["abstract"].astype(str)
        self.embeddings = self.model.encode(texts, batch_size=256, convert_to_numpy=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(GTR_INDEX_PATH):
            try:
                with open(GTR_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached GTR embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(GTR_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        query_embedding = self.model.encode([query], convert_to_numpy=True)
        scores = query_embedding @ self.embeddings.T
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

# ✅ **Ensure Retrieval Occurs Before Query Reformulation**
gtr_retriever = GTRRetriever(corpus_df)

if gtr_retriever.embeddings is None:
    print("⚠️ GTR embeddings not found, computing...")
    gtr_retriever._prepare_embeddings()
class GRITRetriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = GritLM("GritLM/GritLM-7B", device_map="auto", torch_dtype=torch.bfloat16, mode="embedding")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save GRITLM embeddings if not cached."""
        if self._try_load_cache():
            return

        print("🔄 Computing GRIT embeddings...")
        texts = [f"<|embed|>\n{text}" for text in self.corpus_df["abstract"].astype(str)]
        self.embeddings = self.model.encode(texts, batch_size=128, convert_to_numpy=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(GRIT_INDEX_PATH):
            try:
                with open(GRIT_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached GRIT embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(GRIT_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        formatted_query = f"<|user|>\nRepresent this query for retrieving relevant documents:\n<|embed|>\n{query}"
        query_embedding = self.model.encode([formatted_query], convert_to_numpy=True)
        scores = query_embedding @ self.embeddings.T
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

# ✅ **Ensure Retrieval Occurs Before Query Reformulation**
grit_retriever = GRITRetriever(corpus_df)

if grit_retriever.embeddings is None:
    print("⚠️ GRITLM embeddings not found, computing...")
    grit_retriever._prepare_embeddings()

✅ Loaded cached E5 embeddings
✅ Loaded cached GTR embeddings


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn
✅ Loaded cached GRIT embeddings


In [ ]:
for strategy in ["zero-shot", "few-shot", "chain-of-thought (Standard)","chain-of-thought (Neutral)","chain-of-thought (Praise)",
                 "chain-of-thought (Punishment)",
                "chain-of-verification","chain-of-citation","chain-of-quote","Academic Aspiration (2-stage)",
                 "chain-of-thought (Punishment 2 stage)","chain-of-thought (Praise 2 stage)","self-prompt-tuning (role-gen)","self-prompt-tuning","step-back prompting"]:

In [11]:
def format_context(retrieved_docs):
    if isinstance(retrieved_docs, pd.DataFrame):
        return "\n".join([
            f"Corpus ID: {row['corpusid']}\nTitle: {row['title']}\nAbstract: {row['abstract']}"
            for _, row in retrieved_docs.iterrows()
        ])
    return ""

In [12]:


import os
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
from groq import Groq

# ✅ **Initialize Retrievers**
retrievers = {
    #"E5": e5_retriever,
    #"GTR": gtr_retriever,
    "GRITLM": grit_retriever
}

# ✅ **Ensure Ground-Truth IDs are Correctly Formatted**
query_df["corpusids"] = query_df["corpusids"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# ✅ **Sample Only 10% of the Dataset**
query_sample_df = query_df.sample(frac=0.5, random_state=42)  # Randomly select 10% of rows

# ✅ **Run Retrieval, Then Reformulate Query and Run RAG**
rag_results = []

for retriever_name, retriever in retrievers.items():
    print(f"\n🚀 Running Retrieval with {retriever_name} on 10% dataset...\n")

    for _, row in tqdm(query_sample_df.iterrows(), total=len(query_sample_df), desc=f"Retrieving with {retriever_name}"):
        original_query = row["query"]
        ground_truth_corpus_ids = row["corpusids"]

        # **Step 1: Retrieve First, Before Query Reformulation**
        retrieved_ids = retriever.retrieve(original_query, k=20)
        retrieved_ids_for_llm = retrieved_ids[:5]
        
        # **Fetch retrieved document details**
        ranked_retrieved_df = pd.DataFrame({"corpusid": retrieved_ids})
        retrieved_docs_info = ranked_retrieved_df.merge(
            corpus_df[["corpusid", "title", "abstract"]], 
            on="corpusid", 
            how="left"
        )
        retrieved_corpus_ids = retrieved_docs_info["corpusid"].tolist()
        retrieved_titles = retrieved_docs_info["title"].tolist()
        retrieved_abstracts = retrieved_docs_info["abstract"].tolist()

        # **Step 2: Reformulate Query AFTER Retrieval**
        #for strategy in ["zero-shot", "few-shot", "chain-of-thought (Standard)","chain-of-thought (Neutral)","chain-of-thought (Praise)","chain-of-thought (Punishment)"]:
        for strategy in ["Instruction + Role + Motivation (Negative) + Example","Instruction + Role + Motivation (Positive) + Example","zero-shot", "few-shot", "chain-of-thought (Standard)","Role","Motivation (Negative)","Motivation (Positive)","Role + Motivation (Negative)","Role + Motivation (Positive)","Instruction + Role + Motivation (Negative)","Instruction + Role + Motivation (Positive)"]:   
            modified_query = reformulate_query(original_query, strategy)

            # **Step 3: Run LLM with Retrieved Documents**
            generated_answer = generate_with_rag(original_query, modified_query, retrieved_ids_for_llm, retrieved_titles[:5], retrieved_abstracts[:5])

            # **Step 4: Extract Cited Corpus IDs from LLM Response**
            cited_corpus_ids = [str(doc_id) for doc_id in retrieved_ids_for_llm if str(doc_id) in generated_answer]

            # **Step 5: Store Results**
            rag_results.append({
                "Retriever": retriever_name,
                "Query": original_query,
                "Strategy": strategy,
                "Retrieved Corpus IDs": retrieved_corpus_ids,
                "Retrieved Abstracts": retrieved_abstracts,
                "Generated Answer": generated_answer,
                "Cited Corpus IDs": cited_corpus_ids,
                "Ground Truth Corpus IDs": ground_truth_corpus_ids
            })

# ✅ Convert Results to DataFrame
rag_results_df = pd.DataFrame(rag_results)

# ✅ Save Results
RAG_RESULTS_PATH = os.path.join(WORKSPACE_PATH, "rag_results_e5_gtr_gritlm_10percent.csv")
rag_results_df.to_csv(RAG_RESULTS_PATH, index=False)
display(rag_results_df)

print(f"✅ Results saved at: {RAG_RESULTS_PATH}")


🚀 Running Retrieval with GRITLM on 10% dataset...



Retrieving with GRITLM: 100%|███████████████████████████████████████████████████████| 298/298 [1:02:43<00:00, 12.63s/it]


,Retriever,Query,Strategy,Retrieved Corpus IDs,Retrieved Abstracts,Generated Answer,Cited Corpus IDs,Ground Truth Corpus IDs
0,GRITLM,Can you recommend research that uses an LLM to...,Instruction + Role + Motivation (Negative) + E...,"[258960666, 245218561, 252596001, 252716013, 2...",[Retrieval augmentation can aid language model...,Here's the answer:\n\nQuestion: Can you recomm...,"[258960666, 252596001, 252716013, 258041372]",[229923710]
1,GRITLM,Can you recommend research that uses an LLM to...,Instruction + Role + Motivation (Positive) + E...,"[258960666, 245218561, 252596001, 252716013, 2...",[Retrieval augmentation can aid language model...,"Based on the retrieved documents, I identified...","[258960666, 252596001, 252716013]",[229923710]
2,GRITLM,Can you recommend research that uses an LLM to...,zero-shot,"[258960666, 245218561, 252596001, 252716013, 2...",[Retrieval augmentation can aid language model...,"Based on the provided corpus IDs, I recommend ...","[258960666, 252596001, 258041372]",[229923710]
3,GRITLM,Can you recommend research that uses an LLM to...,few-shot,"[258960666, 245218561, 252596001, 252716013, 2...",[Retrieval augmentation can aid language model...,"Based on the retrieved documents, I can recomm...","[258960666, 252596001, 252716013]",[229923710]
4,GRITLM,Can you recommend research that uses an LLM to...,chain-of-thought (Standard),"[258960666, 245218561, 252596001, 252716013, 2...",[Retrieval augmentation can aid language model...,"Based on the retrieved documents, I can sugges...","[258960666, 252596001, 252716013]",[229923710]
...,...,...,...,...,...,...,...,...
3571,GRITLM,Could you suggest a study that explores data a...,Motivation (Positive),"[776271, 15960453, 10435668, 49534653, 6452962...",[Anaphoric shell nouns such as this issue and ...,"Based on the provided corpus, I'd like to sugg...",[776271],[245130931]
3572,GRITLM,Could you suggest a study that explores data a...,Role + Motivation (Negative),"[776271, 15960453, 10435668, 49534653, 6452962...",[Anaphoric shell nouns such as this issue and ...,A very specific and important question!\n\nAft...,[15960453],[245130931]
3573,GRITLM,Could you suggest a study that explores data a...,Role + Motivation (Positive),"[776271, 15960453, 10435668, 49534653, 6452962...",[Anaphoric shell nouns such as this issue and ...,What a great question! \n\nAfter reviewing the...,[15960453],[245130931]
3574,GRITLM,Could you suggest a study that explores data a...,Instruction + Role + Motivation (Negative),"[776271, 15960453, 10435668, 49534653, 6452962...",[Anaphoric shell nouns such as this issue and ...,What a great question! As a researcher special...,"[15960453, 10435668, 6452962]",[245130931]


✅ Results saved at: /data/horse/ws/uskh580e-myws/rag_results_e5_gtr_gritlm_10percent.csv


In [2]:
path = "/data/horse/ws/uskh580e-myws/rag_results_e5_gtr_gritlm_10percent.csv"
rag_results_df = pd.read_csv(path)

In [3]:
print(rag_results_df[["Retrieved Corpus IDs", "Ground Truth Corpus IDs", "Cited Corpus IDs"]].head())

                                Retrieved Corpus IDs Ground Truth Corpus IDs  \
0  [258960666, 245218561, 252596001, 252716013, 2...             [229923710]   
1  [258960666, 245218561, 252596001, 252716013, 2...             [229923710]   
2  [258960666, 245218561, 252596001, 252716013, 2...             [229923710]   
3  [258960666, 245218561, 252596001, 252716013, 2...             [229923710]   
4  [258960666, 245218561, 252596001, 252716013, 2...             [229923710]   

                                    Cited Corpus IDs  
0  ['258960666', '252596001', '252716013', '25804...  
1            ['258960666', '252596001', '252716013']  
2            ['258960666', '252596001', '258041372']  
3            ['258960666', '252596001', '252716013']  
4            ['258960666', '252596001', '252716013']  


In [4]:
import pandas as pd
import re

# --- 1) Robust parsing into lists of strings ---
def parse_id_list(cell):
    # If it’s already a Python list, just stringify each element
    if isinstance(cell, list):
        return [str(x) for x in cell]
    # If it’s a string, pull out all digit-runs (handles spaces, commas, no commas, brackets, etc.)
    if isinstance(cell, str):
        return re.findall(r"\d+", cell)
    # Anything else (NaN, None, etc.) → empty list
    return []

for col in [
    "Retrieved Corpus IDs",
    "Ground Truth Corpus IDs",
    "Cited Corpus IDs"
]:
    rag_results_df[col] = rag_results_df[col].apply(parse_id_list)

# Optional sanity check:
print(rag_results_df.loc[0, [
    "Retrieved Corpus IDs",
    "Ground Truth Corpus IDs",
    "Cited Corpus IDs"
]])

# --- 2) Compute metrics per row ---
def compute_metrics(row):
    retrieved = row["Retrieved Corpus IDs"][:20]
    ground    = set(row["Ground Truth Corpus IDs"])
    cited     = set(row["Cited Corpus IDs"])

    tp5   = len(set(retrieved[:5]) & ground)
    tp20  = len(set(retrieved)     & ground)
    tpc   = len(cited              & ground)

    recall5  = tp5  / len(ground) if ground else 0.0
    recall20 = tp20 / len(ground) if ground else 0.0
    prec     = tpc  / len(cited)  if cited  else 0.0
    rec      = tpc  / len(ground) if ground else 0.0

    return pd.Series({
        "Recall@5":           recall5,
        "Recall@20":          recall20,
        "Citation Precision": prec,
        "Citation Recall":    rec
    })

metrics_df = rag_results_df.apply(compute_metrics, axis=1)
rag_results_df = pd.concat([rag_results_df, metrics_df], axis=1)

# --- 3) Aggregate by strategy ---
# Note: your DataFrame uses column "Strategy" (capital S) not "strategy"
metrics_summary = (
    rag_results_df
    .groupby("Strategy")[
        ["Recall@5","Recall@20","Citation Precision","Citation Recall"]
    ]
    .mean()
    .reset_index()
)

display(metrics_summary)



Retrieved Corpus IDs       [258960666, 245218561, 252596001, 252716013, 2...
Ground Truth Corpus IDs                                          [229923710]
Cited Corpus IDs                [258960666, 252596001, 252716013, 258041372]
Name: 0, dtype: object


,Strategy,Recall@5,Recall@20,Citation Precision,Citation Recall
0,Instruction + Role + Motivation (Negative),0.598434,0.749441,0.308166,0.531879
1,Instruction + Role + Motivation (Negative) + E...,0.598434,0.749441,0.284787,0.552013
2,Instruction + Role + Motivation (Positive),0.598434,0.749441,0.309172,0.538031
3,Instruction + Role + Motivation (Positive) + E...,0.598434,0.749441,0.271644,0.538031
4,Motivation (Negative),0.598434,0.749441,0.290324,0.522931
5,Motivation (Positive),0.598434,0.749441,0.300615,0.516219
6,Role,0.598434,0.749441,0.311130,0.532998
7,Role + Motivation (Negative),0.598434,0.749441,0.301007,0.527964
8,Role + Motivation (Positive),0.598434,0.749441,0.301678,0.531879
9,chain-of-thought (Standard),0.598434,0.749441,0.287025,0.531879


In [2]:
import os
import pickle
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
from groq import Groq
from sentence_transformers import SentenceTransformer
from gritlm import GritLM

# Configuration
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
GROQ_API_KEY = "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"
os.makedirs(WORKSPACE_PATH, exist_ok=True)

# Base Retriever Class (for shared functionality)
class BaseRetriever:
    def __init__(self, corpus_df, text_column="abstract"):
        self.corpus_df = corpus_df
        self.text_column = text_column
        self.embeddings = None
        self._prepare_embeddings()
        
    def _content_hash(self):
        """Generate unique hash for corpus state"""
        content = self.corpus_df[self.text_column].astype(str).sum()
        metadata = str(self.corpus_df.index.tolist())
        return hash(f"{content}|{metadata}")
    
    def _prepare_embeddings(self): 
        raise NotImplementedError
        
    def retrieve(self, query, top_k=5):
        raise NotImplementedError

# E5 Retriever Implementation
class E5Retriever(BaseRetriever):
    def __init__(self, corpus_df):
        self.model_name = "intfloat/e5-large-v2"
        self.cache_file = os.path.join(WORKSPACE_PATH, "e5_embeddings.pkl")
        super().__init__(corpus_df)
        
    def _prepare_embeddings(self):
        # Try loading cached embeddings
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, "rb") as f:
                    stored_hash, embeddings = pickle.load(f)
                    if stored_hash == self._content_hash():
                        self.embeddings = embeddings
                        print("✅ Loaded cached E5 embeddings")
                        return
            except Exception as e:
                print(f"⚠️ E5 cache load failed: {e}")
                
        # Compute fresh embeddings
        print("🔄 Computing E5 embeddings...")
        model = SentenceTransformer(self.model_name, 
                                  device="cuda" if torch.cuda.is_available() else "cpu")
        
        texts = ["passage: " + text for text in self.corpus_df[self.text_column].astype(str)]
        self.embeddings = model.encode(
            texts, 
            batch_size=256, 
            normalize_embeddings=True,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        
        # Save to cache
        with open(self.cache_file, "wb") as f:
            pickle.dump((self._content_hash(), self.embeddings), f)
            
    def retrieve(self, query, top_k=5):
        model = SentenceTransformer(self.model_name)
        query = "query: " + query
        query_embedding = model.encode([query], normalize_embeddings=True)
        scores = query_embedding @ self.embeddings.T
        indices = np.argsort(-scores[0])[:top_k]
        return self.corpus_df.iloc[indices].to_dict(orient='records')

# GTR Retriever Implementation
class GTRRetriever(BaseRetriever):
    def __init__(self, corpus_df):
        self.model_name = "sentence-transformers/gtr-t5-large"
        self.cache_file = os.path.join(WORKSPACE_PATH, "gtr_embeddings.pkl")
        super().__init__(corpus_df)
        
    def _prepare_embeddings(self):
        # Try loading cached embeddings
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, "rb") as f:
                    stored_hash, embeddings = pickle.load(f)
                    if stored_hash == self._content_hash():
                        self.embeddings = embeddings
                        print("✅ Loaded cached GTR embeddings")
                        return
            except Exception as e:
                print(f"⚠️ GTR cache load failed: {e}")
                
        # Compute fresh embeddings
        print("🔄 Computing GTR embeddings...")
        model = SentenceTransformer(self.model_name, 
                                  device="cuda" if torch.cuda.is_available() else "cpu")
        
        self.embeddings = model.encode(
            self.corpus_df[self.text_column].astype(str),
            batch_size=256,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        
        # Save to cache
        with open(self.cache_file, "wb") as f:
            pickle.dump((self._content_hash(), self.embeddings), f)
            
    def retrieve(self, query, top_k=5):
        model = SentenceTransformer(self.model_name)
        query_embedding = model.encode([query])
        scores = query_embedding @ self.embeddings.T
        indices = np.argsort(-scores[0])[:top_k]
        return self.corpus_df.iloc[indices].to_dict(orient='records')

# GRIT Retriever Implementation
class GRITRetriever(BaseRetriever):
    def __init__(self, corpus_df):
        self.model_name = "GritLM/GritLM-7B"
        self.instruction = "Represent this query for retrieving relevant documents:"
        self.cache_file = os.path.join(WORKSPACE_PATH, "grit_embeddings.pkl")
        super().__init__(corpus_df)
        
    def _prepare_embeddings(self):
        # Try loading cached embeddings
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, "rb") as f:
                    stored_hash, embeddings = pickle.load(f)
                    if stored_hash == self._content_hash():
                        self.embeddings = embeddings
                        print("✅ Loaded cached GRIT embeddings")
                        return
            except Exception as e:
                print(f"⚠️ GRIT cache load failed: {e}")
                
        # Compute fresh embeddings
        print("🔄 Computing GRIT embeddings... (This may take a while)")
        model = GritLM(self.model_name,
                      torch_dtype=torch.bfloat16,
                      device_map="auto",
                      mode="embedding")
        
        texts = [f"<|embed|>\n{text}" for text in self.corpus_df[self.text_column].astype(str)]
        self.embeddings = model.encode(
            texts,
            batch_size=128,  # Reduced for stability
            show_progress_bar=True,
            convert_to_numpy=True
        )
        # Handle NaN values
        self.embeddings = np.nan_to_num(self.embeddings, nan=0.0)
        
        # Save to cache
        with open(self.cache_file, "wb") as f:
            pickle.dump((self._content_hash(), self.embeddings), f)
            
    def retrieve(self, query, top_k=5):
        model = GritLM(self.model_name, mode="embedding")
        formatted_query = f"<|user|>\n{self.instruction}\n<|embed|>\n{query}"
        query_embedding = model.encode([formatted_query], convert_to_numpy=True)
        scores = query_embedding @ self.embeddings.T
        indices = np.argsort(-scores[0])[:top_k]
        return self.corpus_df.iloc[indices].to_dict(orient='records')

# RAG Evaluation Pipeline
class LitSearchEvaluator:
    def __init__(self):
        self.groq = Groq(api_key=GROQ_API_KEY)
        self.retrievers = {
            #"e5": E5Retriever(corpus_df),
            #"gtr": GTRRetriever(corpus_df),
            "grit": GRITRetriever(corpus_df)
        }
        
    def reformulate_query(self, query, strategy):
        if strategy == "zero-shot":
            return query
        elif strategy == "few-shot":
            return f"""Answer with citations like (Document ID: X). Examples:
            Question: "Transformer models"
            Answer: "Use self-attention mechanisms (Document ID: 123). 
                     Outperform RNNs on long sequences (Document ID: 456)."
            Now answer: {query}"""
        elif strategy == "chain-of-thought":
            return f"""Analyze step-by-step:
            1. Key concepts in: "{query}"
            2. Match to Document IDs
            3. Cite sources as (Document ID: X)
            Final Answer:"""
        return query
    
    def generate_answer(self, query, context_docs):
        if not context_docs:
            return "No relevant documents found"
            
        context = "RETRIEVED DOCUMENTS:\n" + "\n\n".join([
            f"Document ID: {doc['corpusid']}\nTitle: {doc.get('title', '')}\nContent: {doc['abstract'][:200]}..."
            for doc in context_docs
        ])
        
        response = self.groq.chat.completions.create(
            messages=[{
                "role": "system",
                "content": "You are a research assistant. Cite sources using (Document ID: X) format."
            }, {
                "role": "user",
                "content": f"{context}\n\nQuestion: {query}\nAnswer:"
            }],
            model="llama3-8b-8192",
            temperature=0,
            max_tokens=512
        )
        return response.choices[0].message.content
    
    def extract_citations(self, text):
        citations = re.findall(r'Document ID: (\d+)', text)
        return list(map(int, citations))
    
    def evaluate_retriever(self, retriever_name, strategies=None, sample_frac=0.1):
        """Evaluate a single retriever with multiple strategies"""
        if retriever_name not in self.retrievers:
            print(f"⚠️ Retriever {retriever_name} not available")
            return pd.DataFrame()
            
        strategies = strategies or ["zero-shot", "few-shot", "chain-of-thought"]
        results = []
        retriever = self.retrievers[retriever_name]
        
        # Use a sample of queries
        query_sample = query_df.sample(frac=sample_frac, random_state=42)
        
        for _, row in tqdm(query_sample.iterrows(), desc=f"Evaluating {retriever_name}"):
            ground_truth = set(row["corpusids"])
            query = row["query"]
            
            for strategy in strategies:
                modified_query = self.reformulate_query(query, strategy)
                retrieved_docs = retriever.retrieve(modified_query)
                answer = self.generate_answer(modified_query, retrieved_docs)
                citations = self.extract_citations(answer)
                
                # Calculate metrics
                tp = len(set(citations) & ground_truth)
                precision = tp / len(citations) if citations else 0
                recall = tp / len(ground_truth) if ground_truth else 0
                f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0
                
                results.append({
                    "retriever": retriever_name,
                    "strategy": strategy,
                    "query": query,
                    "citations": citations,
                    "ground_truth": list(ground_truth),
                    "precision": precision,
                    "recall": recall,
                    "f1": f1,
                    "answer": answer
                })
                
        return pd.DataFrame(results)

# Main Execution
if __name__ == "__main__":
    # Load your datasets (replace with actual paths)
    # corpus_df = pd.read_csv("corpus.csv")
    # query_df = pd.read_csv("queries.csv")
    
    # Initialize evaluator
    evaluator = LitSearchEvaluator()
    
    # Evaluate all retrievers
    all_results = []
    for retriever_name in ["grit"]:
        results_df = evaluator.evaluate_retriever(retriever_name, sample_frac=0.1)
        if not results_df.empty:
            results_df.to_csv(os.path.join(WORKSPACE_PATH, f"{retriever_name}_results.csv"), index=False)
            all_results.append(results_df)
    
    # Combine and analyze results
    if all_results:
        full_results = pd.concat(all_results)
        full_results.to_csv(os.path.join(WORKSPACE_PATH, "all_results.csv"), index=False)
        
        # Performance summary
        summary = full_results.groupby(["retriever", "strategy"]).agg({
            "precision": "mean",
            "recall": "mean",
            "f1": "mean"
        }).reset_index()
        
        print("\n=== PERFORMANCE SUMMARY ===")
        print(summary)
        
        # Save summary
        summary.to_csv(os.path.join(WORKSPACE_PATH, "performance_summary.csv"), index=False)
        
        # Sample output
        sample = full_results.iloc[0]
        print("\n=== SAMPLE RESULT ===")
        print(f"Retriever: {sample['retriever']}")
        print(f"Strategy: {sample['strategy']}")
        print(f"Query: {sample['query']}")
        print(f"Answer Excerpt: {sample['answer'][:200]}...")
        print(f"Cited IDs: {sample['citations']}")
        print(f"Ground Truth: {sample['ground_truth']}")
        print(f"F1 Score: {sample['f1']:.2f}")
    else:
        print("No results generated")

NameError: name 'corpus_df' is not defined

[nltk_data] Downloading package punkt to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...


Workspace set up at: /data/horse/ws/uskh580e-myws


Datasets loaded successfully and stored in: /data/horse/ws/uskh580e-myws/huggingface_datasets


In [10]:
query_df.head()

,query_set,query,specificity,quality,corpusids
0,inline_acl,Are there any research papers on methods to co...,0,2,[202719327]
1,inline_acl,Are there any resources available for translat...,1,2,[227231792]
2,inline_acl,Are there any studies that explore post-hoc te...,0,2,"[226254579, 204976362]"
3,inline_acl,Are there any tools or studies that have focus...,1,2,"[10961392, 12160022]"
4,inline_acl,Are there papers that propose contextualized c...,1,2,[233296182]


In [9]:
class GRITRetriever:
    def __init__(self, corpus_df, model_name="GritLM/GritLM-7B"):
        self.corpus_df = corpus_df
        self.model_name = model_name
        self.cache_file = os.path.join(WORKSPACE_PATH, "grit_embeddings.pkl")
        self.embeddings = None
        self._prepare_embeddings()
    
    def _prepare_embeddings(self):
        if self._try_load_cache():
            return
            
        print("🔄 Computing GRIT embeddings...")
        model = GritLM(self.model_name, 
                      torch_dtype=torch.bfloat16,
                      device_map="auto",
                      mode="embedding")

        texts = [f"<|embed|>\n{text}" for text in self.corpus_df["abstract"].astype(str)]
        
        self.embeddings = model.encode(
            texts,
            batch_size=128,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        # Normalize embeddings
        norms = np.linalg.norm(self.embeddings, axis=1, keepdims=True)
        norms[norms == 0] = 1e-10  # Avoid division by zero
        self.embeddings = self.embeddings / norms
        self._save_cache()

    def _try_load_cache(self):
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, "rb") as f:
                    stored_hash, embeddings = pickle.load(f)
                    self.embeddings = embeddings
                    print("✅ Loaded cached GRIT embeddings")
                    return True
            except Exception as e:
                print(f"⚠️ Cache load failed: {e}")
        return False

    def _save_cache(self):
        with open(self.cache_file, "wb") as f:
            pickle.dump((None, self.embeddings), f)
            
    def filter_duplicates(self, docs):
        """Ensure diversity in retrieved documents"""
        seen_ids = set()
        unique_docs = []
        for doc in docs:
            if doc['corpusid'] not in seen_ids:
                seen_ids.add(doc['corpusid'])
                unique_docs.append(doc)
        return unique_docs[:5]  # Return top 5 unique

    def retrieve(self, query, top_k=10):  # Retrieve more then filter
        model = GritLM(self.model_name, mode="embedding")
        formatted_query = f"<|user|>\nRepresent this query for retrieving relevant documents:\n<|embed|>\n{query}"
        query_embedding = model.encode([formatted_query], convert_to_numpy=True)
        
        # Normalize query embedding
        query_embedding = query_embedding / np.linalg.norm(query_embedding)
        
        # Calculate cosine similarity
        scores = query_embedding @ self.embeddings.T
        indices = np.argsort(-scores[0])[:top_k]
        retrieved = self.corpus_df.iloc[indices].to_dict(orient='records')
        return self.filter_duplicates(retrieved)


In [10]:
def generate_answer(query, retrieved_docs):
    # Return early if no documents
    if not retrieved_docs:
        return "No relevant documents found for this query."
    
    context = "RELEVANT DOCUMENTS:\n" + "\n---\n".join([
        f"[Document {doc['corpusid']}] {doc.get('title', 'Untitled')}\nAbstract: {doc['abstract'][:200]}..."
        for doc in retrieved_docs
    ])
    
    response = groq_client.chat.completions.create(
        messages=[{
            "role": "system",
            "content": """You are a precise research assistant. Follow these rules:
            1. Base answers ONLY on provided documents
            2. Cite sources as (Corpus ID: X) after each claim
            3. If document conflicts, mention both
            4. For unsupported claims, say "No supporting evidence"
            5. Keep citations concise and relevant"""
        }, {
            "role": "user",
            "content": f"{context}\n\nQUESTION: {query}\nANSWER:"
        }],
        model="llama3-8b-8192",
        temperature=0,  # Balanced for creativity/accuracy
        max_tokens=512,
        stop=["\n\n", "###"]
    )
    return response.choices[0].message.content

# Robust citation extraction
def extract_citations(text):
    patterns = [
        r'\(Corpus ID: (\d+)\)', 
        r'Corpus ID: (\d+)',
        r'Source: (\d+)',
        r'\[(\d+)\]'
    ]
    citations = []
    for pattern in patterns:
        citations.extend(re.findall(pattern, text))
    return list(map(int, citations))


In [14]:
class BM25Retriever:
    def __init__(self, corpus_df, text_column="abstract", stopword_removal=True, use_stemming=True):
        """
        BM25 Retriever for document retrieval.

        Parameters:
        - corpus_df: Pandas DataFrame containing the corpus
        - text_column: Column in corpus_df that contains the text for retrieval
        - stopword_removal: Whether to remove stopwords
        - use_stemming: Whether to apply stemming
        """
        self.corpus_df = corpus_df
        self.text_column = text_column
        self.stopword_removal = stopword_removal
        self.use_stemming = use_stemming
        self.bm25 = None
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words("english"))

    def tokenize_and_stem(self, text):
        """ Tokenize, remove stopwords, and apply stemming (if enabled). """
        tokens = word_tokenize(text.lower())  # Tokenize & lowercase
        if self.stopword_removal:
            tokens = [token for token in tokens if token not in self.stop_words]  # Remove stopwords
        if self.use_stemming:
            tokens = [self.stemmer.stem(token) for token in tokens]  # Apply stemming
        return tokens
    def create_index(self):
        """ Create BM25 index from the corpus. """
        print("Tokenizing and processing corpus...")
        self.corpus_df["tokens"] = self.corpus_df[self.text_column].astype(str).apply(self.tokenize_and_stem)
        self.bm25 = BM25Okapi(self.corpus_df["tokens"].tolist())

    def save_index(self, filename=BM25_INDEX_PATH):
        """ Save BM25 index to workspace. """
        with open(filename, "wb") as f:
            pickle.dump(self.bm25, f)
        print(f"BM25 index saved to {filename}")

    def load_index(self, filename=BM25_INDEX_PATH):
        """ Load BM25 index from workspace. """
        with open(filename, "rb") as f:
            self.bm25 = pickle.load(f)
        print(f"BM25 index loaded from {filename}")

    def retrieve(self, query, k=10):
        """ Retrieve top-k documents for a query. """
        tokenized_query = self.tokenize_and_stem(query)
        scores = self.bm25.get_scores(tokenized_query)
        ranked_indices = np.argsort(scores)[::-1][:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [15]:
query_df["corpusids"] = query_df["corpusids"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [18]:
!pip install GritLM

You should consider upgrading via the '/data/horse/ws/uskh580e-llm_citation_ws/myenv/bin/python -m pip install --upgrade pip' command.


In [20]:
import gritlm

In [21]:
grit_retriever = GRITRetriever(corpus_df)

🔄 Computing GRIT embeddings...


NameError: name 'GritLM' is not defined

In [15]:
# ✅ **Initialize Retrievers**
grit_retriever = GRITRetriever(corpus_df)
retrievers = {
    #"E5": e5_retriever,
    #"GTR": gtr_retriever,
    "GRITLM": grit_retriever
}

🔄 Computing GRIT embeddings...


NameError: name 'GritLM' is not defined

In [14]:
def generate_with_rag(original_query, modified_query, retrieved_corpus_ids, retrieved_titles, retrieved_texts):
    """ Generates a concise answer using RAG, citing only the top 1–2 relevant sources. """
    context = "\n".join([
        f"Corpus ID: {doc_id}\nTitle: {title}\nFull Text: {full_text}"
        for doc_id, title, full_text in zip(retrieved_corpus_ids, retrieved_titles, retrieved_texts)
    ])

    prompt = f"""
Retrieved Documents:
{context}

Question: {modified_query}
Answer:"""

    response = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": ""},
            {"role": "user", "content": prompt}
        ]
    )


    return response.choices[0].message.content

In [13]:
def reformulate_query(query, strategy):
    """ Reformulates the query only for LLM input, not for retrieval. """

    if strategy == "zero-shot":
        return query

    elif strategy == "few-shot":
        return f"""Here are examples of how to answer questions while citing sources from retrieved documents.
        ...
        Now, using the retrieved documents below, answer the question while citing sources explicitly:
        {query}"""

    elif strategy == "chain-of-thought (Standard)":
        return f"""Let's break down the question step by step:
        ...
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Neutral)":
        return f"""You are an AI assistant tasked with answering a research question...
        ...
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Praise)":
        return f"""You are an AI assistant known for your high-quality answers...
        ...
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Punishment)":
        return f"""You are an AI assistant whose output will be evaluated...
        ...
        Now, process this query:
        {query}"""

    elif strategy == "chain-of-thought (Academic Aspiration)":
        return f"""You are a rising academic in the field of AI...
        ...
        Now, process this query:
        {query}"""

    elif strategy == "self-verification":
        return f"""
        Step 1: Generate an initial answer based on retrieved documents.
        Step 2: Verify each claim and ensure that citations match the retrieved evidence.
        Step 3: If a claim is unsupported, explicitly state that it cannot be verified.

        Now, answer the following question:
        Question: {query}
        Answer:
        """

    elif strategy == "chain-of-verification":
        return f"""
        Let's break this down into a verification process:
        ...
        Now, answer the following question:
        Question: {query}
        Answer:
        """

    elif strategy == "chain-of-citation":
        return f"""Answer the question step by step, citing the relevant corpus ID after each reasoning step.
Use this format: [corpus: document_id]

Now, begin reasoning:
Question: {query}
Answer:"""

    elif strategy == "chain-of-quote":
        return f"""Answer the question step by step, including a short direct quote from the supporting document for each reasoning step.
Use this format: “quoted sentence” [corpus: document_id]

Now, begin reasoning:
Question: {query}
Answer:"""
